# Allgemine Analyse

*Kai, Andrew und Fabian*

---

## 1. Bestimmung der Integrationsordnung

- Augmented Dickey-Fuller Test (ADF-Test)
- Phillips-Perron Test (PP-Test)
- Kwiatkowski-Phillips-Schmidt-Shin Test (KPSS-Test)


## 2. Transformation zur Stationarität

- Differenzierung
- Erste Differenz: $y_t - y_{t-1}$
- Zweite Differenz: $(y_t - y_{t-1}) - (y_{t-1} - y_{t-2})$
- Logarithmische Transformation
- Moving Average
- Simple Exponential Smoothing
- HP-Filter (Hodrick-Prescott-Filter)


## 3. ACF und PACF Analyse

- Autokorrelationsfunktion (ACF)
- Partielle Autokorrelationsfunktion (PACF)


## 4. Modellspezifikation

- AR-Modelle (Autoregressive)
- MA-Modelle (Moving Average)
- ARMA-Modelle
- ARIMA-Modelle
- Auto-ARIMA-Modell

In [9]:
########################################################
#--------------------- Librarys -----------------------#
########################################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, kpss
import statsmodels.api as sm
from arch.unitroot import ADF, KPSS, PhillipsPerron
from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
import os

In [19]:
########################################################
#------------------- data Import ----------------------#
########################################################

# Laden der Datensätze mit relativen Pfaden
# Der Pfad "../data" geht einen Ordner zurück und dann in den "data"-Ordner
samsung_data = pd.read_csv(os.path.join("../data", "005930.ks_aktien_daten.csv"))
daimler_data = pd.read_csv(os.path.join("../data", "mbg.de_aktien_daten.csv"))
microsoft_data = pd.read_csv(os.path.join("../data", "msft_aktien_daten.csv"))

---

---

## **1. Bestimmung der Integrationsordnung**

- Augmented Dickey-Fuller Test (ADF-Test)
- Phillips-Perron Test (PP-Test)
- Kwiatkowski-Phillips-Schmidt-Shin Test (KPSS-Test)

In [17]:
########################################################
#-----------------------   1.   -----------------------#
#--------- Bestimmung der Integrationsordnung ---------#
########################################################

def stationaritaets_tests(zeitreihe, zeitreihen_name="Zeitreihe"):
    """
    Führt ADF-, KPSS- und Phillips-Perron-Test für eine Zeitreihe durch,
    alle implementiert mit der arch.unitroot Bibliothek.
    
    Parameter:
    -----------
    zeitreihe : pandas.Series oder array-like
        Die Zeitreihendaten, die auf Stationarität getestet werden sollen
    zeitreihen_name : str, optional
        Name der Zeitreihe für die Übersichtlichkeit der Ausgabe
    
    Rückgabe:
    --------
    dict
        Dictionary mit Testergebnissen mit den Schlüsseln 'adf', 'kpss' und 'pp'
    """
    ergebnisse = {}
    
    # Stellen wir sicher, dass wir mit einer bereinigten Zeitreihe arbeiten (keine NaN-Werte)
    bereinigte_reihe = pd.Series(zeitreihe).dropna()
    
    print(f"\n===== Stationaritätstests für {zeitreihen_name} =====\n")
    
    # ADF-Test mit arch.unitroot
    print("Augmented Dickey-Fuller Test (arch.unitroot):")
    try:
        adf = ADF(bereinigte_reihe)
        adf_ausgabe = pd.Series(
            [adf.stat, adf.pvalue, adf.lags, adf.nobs, 
             adf.critical_values['1%'], adf.critical_values['5%'], adf.critical_values['10%']],
            index=['Test-Statistik', 'p-Wert', 'Verwendete Lags', 'Anzahl der Beobachtungen', 
                   'Kritischer Wert (1%)', 'Kritischer Wert (5%)', 'Kritischer Wert (10%)']
        )
        print(adf_ausgabe)
        if adf.pvalue <= 0.05:
            print("Schlussfolgerung: Nullhypothese wird abgelehnt. Die Zeitreihe ist stationär.\n")
        else:
            print("Schlussfolgerung: Nullhypothese kann nicht abgelehnt werden. Die Zeitreihe ist nicht-stationär.\n")
        ergebnisse['adf'] = adf
    except Exception as e:
        print(f"Fehler beim ADF-Test: {e}\n")
        ergebnisse['adf'] = None
    
    # KPSS-Test mit arch.unitroot
    print("KPSS-Test (arch.unitroot):")
    try:
        kpss_test = KPSS(bereinigte_reihe)
        kpss_ausgabe = pd.Series(
            [kpss_test.stat, kpss_test.pvalue, kpss_test.lags, kpss_test.nobs, 
             kpss_test.critical_values['10%'], kpss_test.critical_values['5%'], 
             kpss_test.critical_values['2.5%'] if '2.5%' in kpss_test.critical_values else None, 
             kpss_test.critical_values['1%']],
            index=['Test-Statistik', 'p-Wert', 'Verwendete Lags', 'Anzahl der Beobachtungen', 
                   'Kritischer Wert (10%)', 'Kritischer Wert (5%)', 'Kritischer Wert (2.5%)', 
                   'Kritischer Wert (1%)']
        )
        print(kpss_ausgabe)
        if kpss_test.pvalue <= 0.05:
            print("Schlussfolgerung: Nullhypothese wird abgelehnt. Die Zeitreihe ist nicht-stationär.\n")
        else:
            print("Schlussfolgerung: Nullhypothese kann nicht abgelehnt werden. Die Zeitreihe ist stationär.\n")
        ergebnisse['kpss'] = kpss_test
    except Exception as e:
        print(f"Fehler beim KPSS-Test: {e}\n")
        ergebnisse['kpss'] = None
    
    # Phillips-Perron-Test mit arch.unitroot
    print("Phillips-Perron-Test (arch.unitroot):")
    try:
        pp = PhillipsPerron(bereinigte_reihe)
        pp_ausgabe = pd.Series(
            [pp.stat, pp.pvalue, pp.lags, pp.nobs, 
             pp.critical_values['1%'], pp.critical_values['5%'], pp.critical_values['10%']],
            index=['Test-Statistik', 'p-Wert', 'Verwendete Lags', 'Anzahl der Beobachtungen', 
                   'Kritischer Wert (1%)', 'Kritischer Wert (5%)', 'Kritischer Wert (10%)']
        )
        print(pp_ausgabe)
        if pp.pvalue <= 0.05:
            print("Schlussfolgerung: Nullhypothese wird abgelehnt. Die Zeitreihe ist stationär.\n")
        else:
            print("Schlussfolgerung: Nullhypothese kann nicht abgelehnt werden. Die Zeitreihe ist nicht-stationär.\n")
        ergebnisse['pp'] = pp
    except Exception as e:
        print(f"Fehler beim Phillips-Perron-Test: {e}\n")
        ergebnisse['pp'] = None
    
    return ergebnisse


########################################################
#------------ Stationaritätstests anwenden ------------#
########################################################

# Annahme: Die Daten sind bereits geladen
# Stellen wir sicher, dass wir die Spalte "Close" für jeden Datensatz verwenden
samsung_reihe = samsung_data["Close"]
daimler_reihe = daimler_data["Close"]
microsoft_reihe = microsoft_data["Close"]

# Tests für jeden Datensatz durchführen
samsung_ergebnisse = stationaritaets_tests(samsung_reihe, "Samsung")
daimler_ergebnisse = stationaritaets_tests(daimler_reihe, "Daimler")
microsoft_ergebnisse = stationaritaets_tests(microsoft_reihe, "Microsoft")


===== Stationaritätstests für Microsoft =====

Augmented Dickey-Fuller Test (arch.unitroot):
Test-Statistik                 4.039838
p-Wert                         1.000000
Verwendete Lags               38.000000
Anzahl der Beobachtungen    9833.000000
Kritischer Wert (1%)          -3.431015
Kritischer Wert (5%)          -2.861834
Kritischer Wert (10%)         -2.566926
dtype: float64
Schlussfolgerung: Nullhypothese kann nicht abgelehnt werden. Die Zeitreihe ist nicht-stationär.

KPSS-Test (arch.unitroot):
Test-Statistik                 8.829016
p-Wert                         0.000100
Verwendete Lags               59.000000
Anzahl der Beobachtungen    9872.000000
Kritischer Wert (10%)          0.347500
Kritischer Wert (5%)           0.461400
Kritischer Wert (2.5%)              NaN
Kritischer Wert (1%)           0.742800
dtype: float64
Schlussfolgerung: Nullhypothese wird abgelehnt. Die Zeitreihe ist nicht-stationär.

Phillips-Perron-Test (arch.unitroot):
Test-Statistik                

---

---

## 2. Transformation zur Stationarität

- Differenzierung
- Erste Differenz: $y_t - y_{t-1}$
- Zweite Differenz: $(y_t - y_{t-1}) - (y_{t-1} - y_{t-2})$
- Logarithmische Transformation
- Moving Average
- Simple Exponential Smoothing
- HP-Filter (Hodrick-Prescott-Filter)

In [10]:
def stationaritaets_transformationen(zeitreihe, name):
    """
    Führt verschiedene Transformationen zur Stationarität auf einer Zeitreihe durch.
    
    Parameter:
    -----------
    zeitreihe : pandas.Series
        Die zu transformierende Zeitreihe
    name : str
        Name der Zeitreihe für die Benennung der Ausgabevariablen
    
    Rückgabe:
    --------
    dict
        Dictionary mit allen transformierten Zeitreihen
    """
    transformiert = {}
    
    # Originale Zeitreihe speichern
    transformiert[f"{name}_original"] = zeitreihe
    
    # 1. Differenzierung
    # Erste Differenz
    transformiert[f"{name}_diff1"] = zeitreihe.diff().dropna()
    
    # Zweite Differenz
    transformiert[f"{name}_diff2"] = transformiert[f"{name}_diff1"].diff().dropna()
    
    # 2. Logarithmische Transformation
    # Prüfen, ob alle Werte > 0 sind für Log-Transformation
    if all(zeitreihe > 0):
        # Log-Transformation
        transformiert[f"{name}_log"] = np.log(zeitreihe)
        
        # 3. Log-Differenz (Wachstumsrate)
        transformiert[f"{name}_log_diff"] = transformiert[f"{name}_log"].diff().dropna()
    else:
        print(f"Log-Transformation für {name} nicht möglich, da negative oder Null-Werte vorhanden sind.")
    
    # 4. Moving Average
    # Wähle eine geeignete Fensterlänge für MA
    window = 20
    transformiert[f"{name}_ma"] = zeitreihe.rolling(window=window).mean().dropna()
    
    # Differenz zum MA
    ma_temp = zeitreihe.rolling(window=window).mean()
    transformiert[f"{name}_ma_diff"] = (zeitreihe - ma_temp).dropna()
    
    # 5. Simple Exponential Smoothing
    # Optimal Alpha durch Optimierung finden
    try:
        optimal_model = SimpleExpSmoothing(zeitreihe).fit(optimized=True)
        ses_optimal = optimal_model.fittedvalues
        transformiert[f"{name}_ses"] = ses_optimal
        transformiert[f"{name}_ses_diff"] = zeitreihe - ses_optimal
    except:
        print(f"SES für {name} konnte nicht berechnet werden.")
    
    # 6. HP-Filter
    # Für tägliche Daten (Aktienkurse)
    lambda_param = 129600
    
    try:
        cycle, trend = hpfilter(zeitreihe, lamb=lambda_param)
        transformiert[f"{name}_hp_trend"] = trend
        transformiert[f"{name}_hp_cycle"] = cycle
    except Exception as e:
        print(f"HP-Filter für {name} konnte nicht angewendet werden: {e}")
    
    return transformiert

# Funktion anwenden auf alle drei Zeitreihen
def alle_zeitreihen_transformieren(samsung_data, daimler_data, microsoft_data):
    # Stellen wir sicher, dass wir mit der Spalte "Close" arbeiten
    samsung_reihe = samsung_data["Close"]
    daimler_reihe = daimler_data["Close"]
    microsoft_reihe = microsoft_data["Close"]
    
    # Transformationen für jede Zeitreihe berechnen
    samsung_trans = stationaritaets_transformationen(samsung_reihe, "samsung")
    daimler_trans = stationaritaets_transformationen(daimler_reihe, "daimler")
    microsoft_trans = stationaritaets_transformationen(microsoft_reihe, "microsoft")
    
    # Alle Transformationen in ein Dictionary zusammenfassen
    alle_transformationen = {
        "samsung": samsung_trans,
        "daimler": daimler_trans,
        "microsoft": microsoft_trans
    }
    
    return alle_transformationen

transformationen = alle_zeitreihen_transformieren(samsung_data, daimler_data, microsoft_data)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [14]:
def teste_vorhandene_transformationen(transformationen):
    """
    Führt Stationaritätstests für bereits transformierte Zeitreihen durch
    
    Parameter:
    -----------
    transformationen : dict
        Dictionary mit transformierten Zeitreihen, wie von alle_zeitreihen_transformieren() zurückgegeben
    """
    # Durchlaufe jede Aktie
    for aktie_name, trans_dict in transformationen.items():
        print(f"\n{'='*50}")
        print(f"TESTS FÜR {aktie_name.upper()}")
        print(f"{'='*50}")
        
        # Durchlaufe jede Transformation dieser Aktie
        for trans_name, zeitreihe in trans_dict.items():
            # Erstelle einen sauberen Anzeigenamen
            display_name = f"{aktie_name} | {trans_name.replace(f'{aktie_name}_', '')}"
            
            print(f"\n{'-'*15} {display_name} {'-'*15}")
            
            # Führe die Stationaritätstests durch
            stationaritaets_tests(zeitreihe, display_name)

teste_vorhandene_transformationen(transformationen)


TESTS FÜR SAMSUNG

--------------- samsung | original ---------------

===== Stationaritätstests für samsung | original =====

Augmented Dickey-Fuller Test (arch.unitroot):
Test-Statistik                -0.694907
p-Wert                         0.848064
Verwendete Lags               28.000000
Anzahl der Beobachtungen    6320.000000
Kritischer Wert (1%)          -3.431385
Kritischer Wert (5%)          -2.861997
Kritischer Wert (10%)         -2.567013
dtype: float64
Schlussfolgerung: Nullhypothese kann nicht abgelehnt werden. Die Zeitreihe ist nicht-stationär.

KPSS-Test (arch.unitroot):
Test-Statistik                11.594775
p-Wert                         0.000100
Verwendete Lags               46.000000
Anzahl der Beobachtungen    6349.000000
Kritischer Wert (10%)          0.347500
Kritischer Wert (5%)           0.461400
Kritischer Wert (2.5%)              NaN
Kritischer Wert (1%)           0.742800
dtype: float64
Schlussfolgerung: Nullhypothese wird abgelehnt. Die Zeitreihe ist nicht-